In [29]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from autogluon.tabular import TabularPredictor

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)

In [31]:
%autoreload
from pipeline import DataPipeline
pipe = DataPipeline()



In [32]:
referance_buildings = pipe.get_reference_buildings_import_data()


In [33]:
referance_buildings
# select these columns from the data
# value	timestamp temperature area wind_speed	wind_direction	cloud_fraction	precipitation

data_for_model = referance_buildings[['value', 'timestamp', 'temperature', 'area', 'wind_speed', 'wind_direction', 'cloud_fraction', 'precipitation']]


In [34]:
import pandas as pd

# Load your dataset
# Assuming you have a DataFrame called 'referance_buildings'
print(data_for_model.size)
data_for_model = data_for_model.dropna()  # Drop missing values if any
print(data_for_model.size)

# Convert timestamp column to datetime and use numerical features
data_for_model['timestamp'] = pd.to_datetime(data_for_model['timestamp'])
# data_for_model['hour'] = data_for_model['timestamp'].dt.hour
# data_for_model['dayofweek'] = data_for_model['timestamp'].dt.dayofweek


# Define the target column and features
target = 'value'

# Split into train and test
train_data = data_for_model.sample(frac=0.8, random_state=42)
test_data = data_for_model.drop(train_data.index)

train_data




No path specified. Models will be saved in: "AutogluonModels/ag-20241016_095842"


Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.13
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 23.6.0: Mon Jul 29 21:14:30 PDT 2024; root:xnu-10063.141.2~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       12.37 GB / 32.00 GB (38.6%)
Disk Space Avail:   653.22 GB / 926.35 GB (70.5%)
No presets specified! To achieve strong results with AutoGluon, it is recommended to use the available presets.
	Recommended Presets (For more details refer to https://auto.gluon.ai/stable/tutorials/tabular/tabular-essentials.html#presets):
	presets='best_quality'   : Maximize accuracy. Default time_limit=3600.
	presets='high_quality'   : Strong accuracy with fast inference speed. Default time_limit=3600.
	presets='good_quality'   : Good accuracy with very fast inference speed. Default time_limit=3600.
	presets='medium_quality' : Fast training time, ideal for

248808
246960
[1000]	valid_set's l1: 5.96737
[2000]	valid_set's l1: 5.65852


KeyboardInterrupt: 

In [ ]:
# Train the model using AutoGluon
predictor = TabularPredictor(label=target, eval_metric='mean_absolute_error').fit(
    train_data, 
    # presets='best_quality'
    )

In [27]:
# Evaluate on test data
performance = predictor.evaluate(test_data)

print("Evaluation Performance:")
print(performance)  # This will show various metrics such as R^2, RMSE, etc.

# To see feature importance
global_importance = predictor.feature_importance(test_data)
print("\nFeature Importance:")
print(global_importance)  # Shows which features had the most impact on model predictions

Computing feature importance via permutation shuffling for 6 features using 5000 rows with 5 shuffle sets...


Evaluation Performance:
{'mean_absolute_error': -6.451102537680954, 'root_mean_squared_error': -8.755899317452531, 'mean_squared_error': -76.6657728573657, 'r2': 0.4564347759561109, 'pearsonr': 0.6812870538718883, 'median_absolute_error': -4.859660873413086}


	22.18s	= Expected runtime (4.44s per shuffle set)
	8.95s	= Actual runtime (Completed 5 of 5 shuffle sets)



Feature Importance:
                importance    stddev       p_value  n  p99_high   p99_low
area              4.005987  0.089819  3.030537e-08  5  4.190925  3.821049
temperature       2.447230  0.053404  2.719677e-08  5  2.557190  2.337269
wind_direction    0.698313  0.023069  1.427140e-07  5  0.745813  0.650814
wind_speed        0.637683  0.032960  8.534659e-07  5  0.705549  0.569817
cloud_fraction    0.380224  0.026137  2.662699e-06  5  0.434041  0.326407
precipitation     0.236724  0.024285  1.310609e-05  5  0.286726  0.186721
